In [1]:
#Q1
! hdfs dfsadmin -fs hdfs://boss:9000 -report | grep 'Live datanodes'

Live datanodes (2):


In [2]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
csv_path = "/nb/hdma-wi-2021.csv"
parquet_path = "/nb/hdma-wi-2021.parquet"
! rm -rf "/nb/hdma-wi-2021.csv"
! wget -O "/nb/hdma-wi-2021.csv" "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"

# Read CSV file and convert to Parquet
table = pv.read_csv(csv_path)
pq.write_table(table, parquet_path)

# Remove existing Parquet files on HDFS
! hdfs dfs -rm -f hdfs://boss:9000/single.parquet
! hdfs dfs -rm -f hdfs://boss:9000/double.parquet

# Copy Parquet files to HDFS with specified block size and replication
! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -copyFromLocal {parquet_path} hdfs://boss:9000/single.parquet
! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -copyFromLocal {parquet_path} hdfs://boss:9000/double.parquet

--2024-10-25 17:58:19--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘/nb/hdma-wi-2021.csv’

/nb/hdma-wi-2021.cs 100%[===================>] 166.84M   106MB/s    in 1.6s    

2024-10-25 17:58:21 (106 MB/s) - ‘/nb/hdma-wi-2021.csv’ saved [174944099/174944099]



In [3]:
#Q2
! hdfs dfs -du -h hdfs://boss:9000/single.parquet
! hdfs dfs -du -h hdfs://boss:9000/double.parquet

15.9 M  15.9 M  hdfs://boss:9000/single.parquet
15.9 M  31.7 M  hdfs://boss:9000/double.parquet


In [4]:
#Q3
import requests
import json

# HDFS URL for the specific file
hdfs_url = "http://boss:9870/webhdfs/v1/single.parquet"

# Parameters for the GETFILESTATUS operation
params = {
    "op": "GETFILESTATUS"
}

# Send GET request to retrieve file status
response = requests.get(hdfs_url, params=params)

# Raise an error if the request was unsuccessful
response.raise_for_status()

# Parse the JSON response
file_status = response.json()
file_status

{'FileStatus': {'accessTime': 1729879115355,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 16642976,
  'modificationTime': 1729879116622,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [5]:
#Q4
import requests

# HDFS URL for the specific file
hdfs_url = "http://boss:9870/webhdfs/v1/single.parquet"

# Parameters for the OPEN operation
params = {
    "op": "OPEN",
    "offset": 0,
    "noredirect": "true"
}

# Send GET request to retrieve the block location
response = requests.get(hdfs_url, params=params)
location = response.json()

location['Location']


'http://93f108f7b020:9864/webhdfs/v1/single.parquet?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [6]:
#Q5
import requests
import re
from collections import defaultdict

def get_block_locations(file_path, total_length, block_size):
    webhdfs_url = f"http://boss:9870/webhdfs/v1{file_path}"
    block_distribution = defaultdict(int)

    # Loop through each block offset
    for offset in range(0, total_length, block_size):
        params = {
            "op": "OPEN",
            "offset": offset,
            "noredirect": "true"
        }
        
        response = requests.get(webhdfs_url, params=params)  
        if response.status_code == 200:
            response_data = response.json()
            location = response_data['Location']
            
            # Extract container ID from the URL using regex
            container_id = re.search(r'http://([^:]+):', location).group(1)
            block_distribution[container_id] += 1

    return dict(block_distribution)

# Fetch file status to get length and block size
status_params = {"op": "GETFILESTATUS"}
response = requests.get("http://boss:9870/webhdfs/v1/single.parquet", params=status_params)

file_status = response.json()['FileStatus']
total_length = file_status['length']
block_size = file_status['blockSize']

# Get block locations and distribution
block_distribution = get_block_locations(file_path="/single.parquet", total_length=total_length, block_size=block_size)
block_distribution


{'93f108f7b020': 9, '4cfc96f7277b': 7}

In [7]:
#Q6
import requests
from collections import defaultdict

def get_block_distribution(file_path):
    webhdfs_url = f"http://boss:9870/webhdfs/v1{file_path}"
    block_distribution = defaultdict(int)

    # Request to get block locations
    params = {
        "op": "GETFILEBLOCKLOCATIONS"
    }
    
    response = requests.get(webhdfs_url, params=params)
    
    if response.status_code == 200:
        block_locations = response.json()['BlockLocations']['BlockLocation']
        
        # Iterate through each block location and count the occurrences of each DataNode
        for block in block_locations:
            for host in block['hosts']:
                block_distribution[host] += 1
    else:
        print(f"Error fetching block locations: {response.status_code} - {response.json()}")
    
    return dict(block_distribution)

# Get distribution of blocks for double.parquet
block_distribution = get_block_distribution("/double.parquet")
block_distribution


{'93f108f7b020': 16, '4cfc96f7277b': 16}

In [8]:
#Q7
import pyarrow as pa
import pyarrow.fs
import pyarrow.parquet as pq
import time
import numpy as np
import pyarrow.compute as pc


# Set HDFS host and port
hdfs_host = "boss"  # replace with your HDFS host
hdfs_port = 9000     # default HDFS port

# Create a HadoopFileSystem instance
hdfs = pa.fs.HadoopFileSystem(hdfs_host, hdfs_port)

# Define the path to the Parquet file in HDFS
parquet_file_path = "/double.parquet" 

# Measure the time taken for reading the file and calculating the mean
start_time = time.time()

# Open the Parquet file
with hdfs.open_input_file(parquet_file_path) as f:
    table = pq.read_table(f)

# Calculate the average loan_amount
average_loan_amount = pc.mean(table["loan_amount"]).as_py()
end_time = time.time()
elapsed_time = end_time - start_time

# Print results
average_loan_amount


2024-10-25 17:59:17,381 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


204961.21752386744

In [13]:
#Q8
# Measure the time taken for reading only the loan_amount column
start_time_q8 = time.time()

# Open the Parquet file and read only the loan_amount column
with hdfs.open_input_file(parquet_file_path) as parquet_file:
    table = pq.read_table(parquet_file, columns=["loan_amount"])

# Calculate the average loan_amount
avg_loan_amount_q8 = pc.mean(table["loan_amount"]).as_py()

end_time_q8 = time.time()
q8_duration = end_time_q8 - start_time_q8

# Calculate and print the speedup ratio
performance_improvement = elapsed_time / q8_duration
performance_improvement

36.16151668852819